# Dots and Boxes

*Wannes Meert, Giuseppe Marra, Pieter Robberechts*  
*Dept CS, KU Leuven*

Code examples for how to play Dots and Boxes as implemented in OpenSpiel.

In [1]:
import random
import pyspiel

## Set up game

In [2]:
num_rows, num_cols = 2, 2  # Number of squares
game_string = (f"dots_and_boxes(num_rows={num_rows},num_cols={num_cols},"
                "utility_margin=true)")
game = pyspiel.load_game(game_string)

In [4]:
params = game.get_parameters()
assert num_rows == params['num_rows']
assert num_cols == params['num_cols']
num_cells = (num_rows + 1) * (num_cols + 1)
num_parts = 3   # (horizontal, vertical, cell)
num_states = 3  # (empty, player1, player2)

def part2num(part):
    p = {'h': 0, 'horizontal': 0,  # Who has set the horizontal line (top of cell)
         'v': 1, 'vertical':   1,  # Who has set the vertical line (left of cell)
         'c': 2, 'cell':       2}  # Who has won the cell
    return p.get(part, part)
def state2num(state):
    s = {'e':  0, 'empty':   0,
         'p1': 1, 'player1': 1,
         'p2': 2, 'player2': 2}
    return s.get(state, state)
def num2state(state):
    s = {0: 'empty', 1: 'player1', 2: 'player2'}
    return s.get(state, state)

print(f"{num_rows=}, {num_cols=}, {num_cells=}")

num_rows=2, num_cols=2, num_cells=9


## Play game

In [5]:
state = game.new_initial_state()
state

┌╴ ╶┬╴ ╶┐
         
├╴ ╶┼╴ ╶┤
         
└╴ ╶┴╴ ╶┘

### Available actions in the current state

In [5]:
current_player = state.current_player()
legal_actions = state.legal_actions()
legal_actions

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [6]:
for action in legal_actions:
      print("Legal action: {} ({})".format(
          state.action_to_string(current_player, action), action))

Legal action: P1(h,0,0) (0)
Legal action: P1(h,0,1) (1)
Legal action: P1(h,1,0) (2)
Legal action: P1(h,1,1) (3)
Legal action: P1(h,2,0) (4)
Legal action: P1(h,2,1) (5)
Legal action: P1(v,0,0) (6)
Legal action: P1(v,0,1) (7)
Legal action: P1(v,0,2) (8)
Legal action: P1(v,1,0) (9)
Legal action: P1(v,1,1) (10)
Legal action: P1(v,1,2) (11)


The formula to switch between action number and action description is based on counting:
- First all horizontal lines, row by row.
- Second all vertical lines, row by row.

In [7]:
action = 10
nb_hlines = (num_rows + 1) * num_cols
if action < nb_hlines:
    row = action // num_cols
    col = action % num_cols
    print(f"{action=} : (h,{row},{col})")
else:
    action2 = action - nb_hlines
    row = action2 // (num_cols + 1)
    col = action2 % (num_cols + 1)
    print(f"{action=} : (v,{row},{col})")

action=10 : (v,1,1)


### Current observation

The observation is expressed as a tensor with dimensions:

- Axis 1: Nb of cellstates: `3` (empty, player1, player2)
- Axis 2: Nb of cells: `(num_rows + 1) * (num_cols + 1)`  
  Cells are counted row-wise (thus `cell = col + row * (num_cols + 1)`
- Axis 3: Nb of cell parts: `3` (horizontal, vertical, won by)

In [8]:
def get_observation(obs_tensor, state, row, col, part):
    state = state2num(state)
    part = part2num(part)
    idx =   part \
          + (row * (num_cols + 1) + col) * num_parts  \
          + state * (num_parts * num_cells)
    return obs_tensor[idx]
def get_observation_state(obs_tensor, row, col, part, as_str=True):
    is_state = None
    for state in range(3):
        if get_observation(obs_tensor, state, row, col, part) == 1.0:
            is_state = state
    if as_str:
        is_state = num2state(is_state)
    return is_state

Set up a game with two steps

In [9]:
state = game.new_initial_state()
state_strs, obs_tensors = [], []
actions = [0, 6]

state_strs += [f"{0:<{num_cols*4+1}}\n" + str(state)]
obs_tensors += [state.observation_tensor()]
for idx, action in enumerate(actions):
    state.apply_action(action)
    state_strs += [f"{idx+1:<{num_cols*4+1}}\n" + str(state)]
    obs_tensors += [state.observation_tensor()]

print("\n".join("   ".join(t) for t in zip(*[s.split("\n") for s in state_strs])))

0           1           2        
┌╴ ╶┬╴ ╶┐   ┌───┬╴ ╶┐   ┌───┬╴ ╶┐
                        │        
├╴ ╶┼╴ ╶┤   ├╴ ╶┼╴ ╶┤   ├╴ ╶┼╴ ╶┤
                                 
└╴ ╶┴╴ ╶┘   └╴ ╶┴╴ ╶┘   └╴ ╶┴╴ ╶┘
      


In [10]:
(get_observation_state(obs_tensors[0], 0, 0, 'h'),
 get_observation_state(obs_tensors[2], 0, 0, 'h'))

('empty', 'player1')

In [11]:
(get_observation_state(obs_tensors[0], 0, 0, 'v'),
 get_observation_state(obs_tensors[2], 0, 0, 'v'))

('empty', 'player2')

## Playing an entire (random) game

Here is a small example that plays an entire (random) game. If you want to see more examples (e.g. including training), open the `open_spiel/python/examples` directory.

In [3]:
state = game.new_initial_state()
print(f"Initial state:")
print(state)
while not state.is_terminal():
    current_player = state.current_player()
    legal_actions = state.legal_actions()
    rand_idx = random.randint(0, len(legal_actions) - 1)
    action = legal_actions[rand_idx]
    state.apply_action(action)
    print(f"Player{current_player+1}:")
    print(state)
returns = state.returns()
print(f"Player return values: {returns}")

Initial state:
┌╴ ╶┬╴ ╶┐
         
├╴ ╶┼╴ ╶┤
         
└╴ ╶┴╴ ╶┘

Player1:
┌╴ ╶┬╴ ╶┐
         
├╴ ╶┼───┤
         
└╴ ╶┴╴ ╶┘

Player2:
┌───┬╴ ╶┐
         
├╴ ╶┼───┤
         
└╴ ╶┴╴ ╶┘

Player1:
┌───┬╴ ╶┐
         
├╴ ╶┼───┤
        │
└╴ ╶┴╴ ╶┘

Player2:
┌───┬╴ ╶┐
         
├╴ ╶┼───┤
    │   │
└╴ ╶┴╴ ╶┘

Player1:
┌───┬╴ ╶┐
│        
├╴ ╶┼───┤
    │   │
└╴ ╶┴╴ ╶┘

Player2:
┌───┬───┐
│        
├╴ ╶┼───┤
    │   │
└╴ ╶┴╴ ╶┘

Player1:
┌───┬───┐
│        
├╴ ╶┼───┤
    │   │
└───┴╴ ╶┘

Player2:
┌───┬───┐
│        
├───┼───┤
    │   │
└───┴╴ ╶┘

Player1:
┌───┬───┐
│ 1 │    
├───┼───┤
    │   │
└───┴╴ ╶┘

Player1:
┌───┬───┐
│ 1 │ 1 │
├───┼───┤
    │   │
└───┴╴ ╶┘

Player1:
┌───┬───┐
│ 1 │ 1 │
├───┼───┤
│ 1 │   │
└───┴╴ ╶┘

Player1:
┌───┬───┐
│ 1 │ 1 │
├───┼───┤
│ 1 │ 1 │
└───┴───┘

Player return values: [4.0, -4.0]


## Running the tournament

You can run the tournament code yourself to test your setup (make sure to also test your code on the server). Check the readme file for more information.

## Play using the GUI

You can run your agent inside a websocket wrapper that is compatible with the user interface 
available on https://github.com/wannesm/dotsandboxes .

Instructions:
- Start agent using `./agent_websocket.py <dir-where-agent-is> 5001`
- Start local server using `./dotsandboxesserver.py 8080` (in the dotsandboxes repo)
- Go to `127.0.0.1:8080`
- Enter `ws://127.0.0.1:5001` as one of the agents
- Start playing

The result should look like:

![GUI](fig/gui-example.png)

## Start the game from a given situation

The situation can be expressed using the Dots-and-Boxes-Notation (DBN). This notation is based on the sequence of actions.

In [20]:
state = game.new_initial_state("100111101111")
state

┌───┬╴ ╶┐
│       │
├╴ ╶┼───┤
│   │   │
└───┴───┘

In [21]:
for action in [2, 1, 7]:
    state.apply_action(action)
    print(state.dbn_string())
    print(state)
print(state.is_terminal())
print(state.returns())

101111101111
┌───┬╴ ╶┐
│       │
├───┼───┤
│ 1 │   │
└───┴───┘

111111101111
┌───┬───┐
│       │
├───┼───┤
│ 1 │   │
└───┴───┘

111111111111
┌───┬───┐
│ 2 │ 2 │
├───┼───┤
│ 1 │   │
└───┴───┘

True
[-1.0, 1.0]
